# Sviluppo di un Chatbot per il Customer Care

Questo notebook illustra come implementare un chatbot semplice ma efficace per il customer care utilizzando tecniche di Natural Language Processing (NLP). Ci concentreremo su un caso d'uso tipico: un assistente virtuale che può rispondere a domande frequenti, guidare gli utenti attraverso processi comuni e raccogliere informazioni prima di un eventuale passaggio a un operatore umano.

## Obiettivi del Notebook

- Comprendere i componenti fondamentali di un chatbot per il customer care
- Implementare un sistema di comprensione del linguaggio naturale (NLU)
- Creare un sistema di gestione del dialogo
- Sviluppare risposte appropriate e contestuali
- Testare il chatbot con scenari realistici di customer care

## Caso d'uso: Chatbot per il Customer Care

Un chatbot per il customer care può:

- Rispondere a domande frequenti 24/7
- Guidare gli utenti attraverso processi come la risoluzione di problemi tecnici
- Raccogliere informazioni preliminari prima di trasferire a un operatore umano
- Gestire richieste semplici come il controllo dello stato di un ordine
- Fornire un'esperienza conversazionale naturale e soddisfacente

In questo notebook, svilupperemo un chatbot che può gestire diversi tipi di richieste nel contesto di un'azienda di telecomunicazioni, con particolare attenzione alle esigenze del customer care.

## 1. Setup e Importazione delle Librerie

In [ ]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import json
import re
import string
from datetime import datetime

# Librerie per NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Librerie per intent recognition
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Impostazioni di visualizzazione
plt.style.use('ggplot')
sns.set(style='whitegrid')

# Download delle risorse NLTK necessarie
nltk.download('punkt')
nltk.download('stopwords')

## 2. Architettura del Chatbot

Il nostro chatbot avrà un'architettura modulare composta da:

1. **Natural Language Understanding (NLU)**: Comprende l'intento dell'utente e estrae entità rilevanti
2. **Dialogue Management**: Gestisce il flusso della conversazione e mantiene lo stato
3. **Response Generation**: Genera risposte appropriate basate sull'intento e sul contesto

Iniziamo definendo gli intenti che il nostro chatbot sarà in grado di gestire.

In [ ]:
# Definiamo gli intenti e gli esempi di frasi per ciascun intento
intents_data = {
    "intents": [
        {
            "tag": "saluto",
            "patterns": [
                "Ciao",
                "Buongiorno",
                "Buonasera",
                "Salve",
                "Ehi",
                "Come va",
                "C'è qualcuno"
            ],
            "responses": [
                "Ciao! Sono l'assistente virtuale di TelcoService. Come posso aiutarti oggi?",
                "Buongiorno! Sono qui per assisterti con i servizi TelcoService. Come posso esserti utile?",
                "Salve! Sono l'assistente virtuale di TelcoService. Cosa posso fare per te oggi?"
            ]
        },
        {
            "tag": "arrivederci",
            "patterns": [
                "Ciao ciao",
                "Arrivederci",
                "A presto",
                "Grazie e buona giornata",
                "Alla prossima",
                "Addio"
            ],
            "responses": [
                "Arrivederci! Non esitare a contattarci nuovamente se hai altre domande.",
                "Grazie per aver contattato TelcoService. Buona giornata!",
                "A presto! Siamo sempre qui per aiutarti."
            ]
        },
        {
            "tag": "ringraziamento",
            "patterns": [
                "Grazie",
                "Grazie mille",
                "Ti ringrazio",
                "Molto utile, grazie",
                "Apprezzo il tuo aiuto"
            ],
            "responses": [
                "Prego, è un piacere poterti aiutare!",
                "Non c'è di che! Sono qui per questo.",
                "Felice di essere stato utile. C'è altro in cui posso aiutarti?"
            ]
        },
        {
            "tag": "problema_connessione",
            "patterns": [
                "La mia connessione internet non funziona",
                "Non riesco a connettermi a internet",
                "La rete è lenta",
                "Il wifi non va",
                "Problemi di connessione",
                "Internet è caduto",
                "Non navigo",
                "La fibra non funziona"
            ],
            "responses": [
                "Mi dispiace per il problema di connessione. Proviamo alcune soluzioni rapide: 1) Riavvia il router spegnendolo, attendendo 30 secondi e riaccendendolo. 2) Verifica che i cavi siano collegati correttamente. 3) Controlla se altri dispositivi riescono a connettersi. Hai già provato qualcuna di queste soluzioni?"
            ]
        },
        {
            "tag": "stato_ordine",
            "patterns": [
                "Dove si trova il mio ordine",
                "Stato della mia spedizione",
                "Quando arriva il mio ordine",
                "Tracking del mio ordine",
                "Informazioni sulla consegna",
                "Il mio ordine è in ritardo",
                "Voglio tracciare il mio ordine"
            ],
            "responses": [
                "Per verificare lo stato del tuo ordine, avrei bisogno del numero d'ordine. Puoi fornirmi il numero d'ordine che trovi nella email di conferma?"
            ]
        },
        {
            "tag": "informazioni_fatturazione",
            "patterns": [
                "Ho una domanda sulla mia fattura",
                "Non capisco un addebito",
                "La mia bolletta è troppo alta",
                "C'è un errore nella fattura",
                "Quando scade la fattura",
                "Come posso pagare la bolletta",
                "Non ho ricevuto la fattura"
            ],
            "responses": [
                "Per questioni relative alla fatturazione, posso aiutarti con informazioni generali. Per verificare dettagli specifici della tua fattura, avrei bisogno del tuo numero cliente o del codice fattura. Preferisci fornirmi questi dati o vuoi informazioni generali sulle modalità di pagamento?"
            ]
        },
        {
            "tag": "cambio_piano",
            "patterns": [
                "Voglio cambiare il mio piano",
                "Opzioni per upgrade",
                "Piani disponibili",
                "Passare a un piano superiore",
                "Offerte attuali",
                "Promozioni in corso",
                "Vorrei più giga",
                "Piano con più minuti"
            ],
            "responses": [
                "Abbiamo diverse opzioni per aggiornare il tuo piano. Attualmente offriamo: 1) Piano Base: 30€/mese con 50GB e chiamate illimitate, 2) Piano Premium: 45€/mese con 100GB e chiamate/SMS illimitati, 3) Piano Unlimited: 60€/mese con dati illimitati e tutti i servizi inclusi. Quale ti interessa maggiormente?"
            ]
        },
        {
            "tag": "assistenza_tecnica",
            "patterns": [
                "Ho bisogno di supporto tecnico",
                "Problemi con il dispositivo",
                "Il mio telefono non funziona",
                "Come configurare la mail",
                "Problemi con l'app",
                "Il decoder non si accende",
                "Errore nel software"
            ],
            "responses": [
                "Per fornirti l'assistenza tecnica più appropriata, avrei bisogno di sapere quale dispositivo o servizio sta causando problemi. Puoi fornirmi maggiori dettagli sul problema che stai riscontrando?"
            ]
        },
        {
            "tag": "orari_negozi",
            "patterns": [
                "Orari di apertura",
                "Quando è aperto il negozio",
                "Orario del centro assistenza",
                "Negozio più vicino",
                "Dove posso trovarvi",
                "Indirizzo del punto vendita",
                "Siete aperti la domenica"
            ],
            "responses": [
                "I nostri negozi sono generalmente aperti dal lunedì al sabato, dalle 9:00 alle 19:30. Alcuni centri commerciali potrebbero avere orari diversi. Per trovare il negozio più vicino a te e verificare gli orari specifici, avrei bisogno di sapere in quale città ti trovi. Puoi indicarmi la tua città?"
            ]
        },
        {
            "tag": "contatto_operatore",
            "patterns": [
                "Voglio parlare con un operatore",
                "Operatore umano per favore",
                "Passa a un agente",
                "Non voglio parlare con un bot",
                "Servono un operatore",
                "Assistenza umana",
                "Posso parlare con una persona vera"
            ],
            "responses": [
                "Capisco che preferisci parlare con un operatore. Per indirizzarti al reparto corretto, potresti dirmi brevemente qual è l'argomento della tua richiesta? (es. problemi tecnici, fatturazione, informazioni commerciali)"
            ]
        },
        {
            "tag": "default",
            "patterns": [],
            "responses": [
                "Mi dispiace, non ho capito completamente la tua richiesta. Puoi riformularla o specificare se hai bisogno di assistenza tecnica, informazioni sulla fatturazione o sui nostri servizi?",
                "Non sono sicuro di aver compreso correttamente. Potresti esprimere la tua richiesta in modo diverso?",
                "Scusa, ma non riesco a interpretare questa richiesta. Posso aiutarti con problemi di connessione, fatturazione, informazioni sui piani tariffari o supporto tecnico. Di cosa hai bisogno?"
            ]
        }
    ]
}

# Salviamo i dati degli intenti in un file JSON per un uso futuro
with open('intents.json', 'w', encoding='utf-8') as f:
    json.dump(intents_data, f, ensure_ascii=False, indent=4)

## 3. Natural Language Understanding (NLU)

Il componente NLU è responsabile di comprendere l'intento dell'utente e estrarre entità rilevanti dal messaggio. Implementiamo un classificatore di intenti utilizzando tecniche di machine learning.

In [ ]:
# Preprocessing del testo
stemmer = SnowballStemmer('italian')
stop_words = set(stopwords.words('italian'))

def preprocess_text(text):
    # Conversione in minuscolo
    text = text.lower()
    
    # Rimozione della punteggiatura
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    
    # Tokenizzazione
    tokens = word_tokenize(text)
    
    # Rimozione delle stopwords e stemming
    processed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words and len(token) > 2]
    
    # Ricostruzione del testo
    processed_text = ' '.join(processed_tokens)
    
    return processed_text

In [ ]:
# Preparazione dei dati per l'addestramento del classificatore di intenti
texts = []
labels = []

for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern)
        labels.append(intent['tag'])

# Creiamo un DataFrame per una migliore visualizzazione
intent_df = pd.DataFrame({'text': texts, 'intent': labels})

# Visualizziamo la distribuzione degli intenti
plt.figure(figsize=(10, 6))
sns.countplot(y='intent', data=intent_df, order=intent_df['intent'].value_counts().index)
plt.title('Distribuzione degli Intenti')
plt.xlabel('Numero di Esempi')
plt.ylabel('Intento')
plt.tight_layout()
plt.show()

In [ ]:
# Preprocessiamo i testi
processed_texts = [preprocess_text(text) for text in texts]

# Dividiamo in training e test set
X_train, X_test, y_train, y_test = train_test_split(processed_texts, labels, test_size=0.2, random_state=42, stratify=labels)

# Creiamo una pipeline con TF-IDF e SVM
intent_classifier = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('svm', LinearSVC(C=10))
])

# Addestriamo il classificatore
intent_classifier.fit(X_train, y_train)

# Valutiamo le performance
y_pred = intent_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Funzione per riconoscere l'intento di un messaggio
def recognize_intent(text):
    processed_text = preprocess_text(text)
    intent = intent_classifier.predict([processed_text])[0]
    
    # Calcoliamo anche un punteggio di confidenza
    # Nota: LinearSVC non fornisce probabilità, quindi usiamo decision_function come approssimazione
    decision_scores = intent_classifier.named_steps['svm'].decision_function(intent_classifier.named_steps['tfidf'].transform([processed_text]))[0]
    confidence = np.max(decision_scores) / np.sum(np.abs(decision_scores))
    
    return {
        'intent': intent,
        'confidence': confidence
    }

# Testiamo la funzione con alcuni esempi
test_messages = [
    "Buongiorno, come posso aiutarti?",
    "La mia connessione internet non funziona da ieri",
    "Vorrei sapere quando arriva il mio ordine",
    "Ho una domanda sulla mia ultima fattura",
    "Voglio parlare con un operatore umano"
]

for message in test_messages:
    result = recognize_intent(message)
    print(f"Messaggio: {message}")
    print(f"Intento riconosciuto: {result['intent']}")
    print(f"Confidenza: {result['confidence']:.4f}\n")

## 4. Estrazione di Entità

Oltre a riconoscere l'intento, è importante estrarre entità rilevanti dal messaggio dell'utente. Implementiamo una semplice funzione per estrarre entità come numeri d'ordine, date, importi, ecc.

In [ ]:
# Funzione per estrarre entità dal testo
def extract_entities(text):
    entities = {}
    
    # Estrazione di numeri d'ordine (formato: #12345 o ORD-12345)
    order_number_pattern = r'#(\d+)|ORD[- ](\d+)'
    order_matches = re.findall(order_number_pattern, text, re.IGNORECASE)
    if order_matches:
        # Prendiamo il primo gruppo non vuoto
        order_number = next(num for match in order_matches for num in match if num)
        entities['order_number'] = order_number
    
    # Estrazione di importi (formato: 10€, €10, 10 euro)
    amount_pattern = r'(\d+)[\s]*[€|euro|EUR]|[€|EUR][\s]*(\d+)'
    amount_matches = re.findall(amount_pattern, text, re.IGNORECASE)
    if amount_matches:
        # Prendiamo il primo gruppo non vuoto
        amount = next(num for match in amount_matches for num in match if num)
        entities['amount'] = amount
    
    # Estrazione di date (formato semplificato: gg/mm/aaaa o gg-mm-aaaa)
    date_pattern = r'(\d{1,2}[/|-]\d{1,2}[/|-]\d{2,4})'
    date_matches = re.findall(date_pattern, text)
    if date_matches:
        entities['date'] = date_matches[0]
    
    # Estrazione di numeri di telefono (formato italiano semplificato)
    phone_pattern = r'(\+39)?[\s|-]?(\d{3})[\s|-]?(\d{3})[\s|-]?(\d{4})'
    phone_matches = re.findall(phone_pattern, text)
    if phone_matches:
        # Ricostruiamo il numero di telefono
        phone_parts = phone_matches[0]
        phone = ''.join(part for part in phone_parts if part)
        entities['phone'] = phone
    
    # Estrazione di città (semplificata, solo alcune città italiane principali)
    cities = ['roma', 'milano', 'napoli', 'torino', 'palermo', 'genova', 'bologna', 'firenze', 'bari', 'catania']
    words = text.lower().split()
    for city in cities:
        if city in words:
            entities['city'] = city.capitalize()
            break
    
    return entities

# Testiamo la funzione con alcuni esempi
test_entity_messages = [
    "Vorrei sapere lo stato del mio ordine #12345",
    "Ho ricevuto una fattura di 59€ ma non capisco per quale servizio",
    "Ho un appuntamento con un tecnico il 15/04/2023, posso spostarlo?",
    "Il mio numero di telefono è +39 123 456 7890",
    "Qual è l'indirizzo del vostro negozio a Milano?"
]

for message in test_entity_messages:
    entities = extract_entities(message)
    print(f"Messaggio: {message}")
    print(f"Entità estratte: {entities}\n")

## 5. Dialogue Management

Il Dialogue Manager è responsabile di gestire il flusso della conversazione e mantenere lo stato del dialogo. Implementiamo un semplice gestore di dialogo basato su stati.

In [ ]:
# Classe per gestire lo stato del dialogo
class DialogueState:
    def __init__(self):
        self.current_intent = None
        self.entities = {}
        self.conversation_history = []
        self.turns = 0
        self.missing_info = []
        self.context = {}
    
    def update(self, user_message, intent, entities):
        # Aggiorniamo lo stato con le nuove informazioni
        self.conversation_history.append({"role": "user", "message": user_message})
        self.turns += 1
        
        # Se l'intento è nuovo, lo aggiorniamo
        if intent != "default" or self.current_intent is None:
            self.current_intent = intent
        
        # Aggiorniamo le entità
        self.entities.update(entities)
        
        # Aggiorniamo le informazioni mancanti in base all'intento
        self._update_missing_info()
    
    def add_bot_response(self, response):
        self.conversation_history.append({"role": "bot", "message": response})
    
    def _update_missing_info(self):
        # Definiamo quali informazioni sono necessarie per ciascun intento
        required_info = {
            "stato_ordine": ["order_number"],
            "informazioni_fatturazione": ["amount"],
            "orari_negozi": ["city"],
            "assistenza_tecnica": ["device_type"],  # Questa entità dovrà essere estratta in modo più sofisticato
            "contatto_operatore": ["issue_type"]  # Questa entità dovrà essere estratta in modo più sofisticato
        }
        
        # Se l'intento corrente richiede informazioni specifiche
        if self.current_intent in required_info:
            # Verifichiamo quali informazioni mancano
            self.missing_info = [info for info in required_info[self.current_intent] 
                               if info not in self.entities]
        else:
            self.missing_info = []
    
    def is_complete(self):
        # Verifichiamo se abbiamo tutte le informazioni necessarie
        return len(self.missing_info) == 0
    
    def get_next_missing_info(self):
        # Restituiamo la prossima informazione mancante
        if self.missing_info:
            return self.missing_info[0]
        return None
    
    def __str__(self):
        return f"Intent: {self.current_intent}, Entities: {self.entities}, Missing: {self.missing_info}, Turns: {self.turns}"

In [ ]:
# Classe per gestire il dialogo
class DialogueManager:
    def __init__(self, intents_data):
        self.intents = {intent['tag']: intent for intent in intents_data['intents']}
        self.state = DialogueState()
    
    def process(self, user_message):
        # Riconosciamo l'intento
        intent_result = recognize_intent(user_message)
        intent = intent_result['intent']
        confidence = intent_result['confidence']
        
        # Se la confidenza è troppo bassa, usiamo l'intento default
        if confidence < 0.3 and intent != "default":
            intent = "default"
        
        # Estraiamo le entità
        entities = extract_entities(user_message)
        
        # Aggiorniamo lo stato del dialogo
        self.state.update(user_message, intent, entities)
        
        # Generiamo la risposta
        response = self._generate_response()
        
        # Aggiungiamo la risposta alla storia della conversazione
        self.state.add_bot_response(response)
        
        return response
    
    def _generate_response(self):
        # Se abbiamo un intento riconosciuto
        if self.state.current_intent in self.intents:
            intent_data = self.intents[self.state.current_intent]
            
            # Se mancano informazioni, chiediamole
            missing_info = self.state.get_next_missing_info()
            if missing_info:
                return self._ask_for_missing_info(missing_info)
            
            # Altrimenti, restituiamo una risposta casuale tra quelle disponibili
            return random.choice(intent_data['responses'])
        
        # Se non abbiamo un intento riconosciuto, usiamo la risposta default
        return random.choice(self.intents['default']['responses'])
    
    def _ask_for_missing_info(self, missing_info):
        # Messaggi per richiedere informazioni mancanti
        info_requests = {
            "order_number": "Per verificare lo stato del tuo ordine, avrei bisogno del numero d'ordine. Puoi fornirmi il numero che trovi nella email di conferma?",
            "amount": "Per aiutarti con la fatturazione, potresti dirmi l'importo della fattura in questione?",
            "city": "In quale città ti trovi? Così posso verificare gli orari del negozio più vicino.",
            "device_type": "Per fornirti l'assistenza tecnica più appropriata, potresti dirmi di quale dispositivo si tratta? (es. smartphone, router, decoder)",
            "issue_type": "Per indirizzarti all'operatore più adatto, potresti specificare il tipo di problema? (es. tecnico, fatturazione, commerciale)"
        }
        
        if missing_info in info_requests:
            return info_requests[missing_info]
        
        # Risposta generica se non abbiamo un messaggio specifico
        return "Mi servono alcune informazioni aggiuntive per aiutarti meglio. Potresti fornirmi più dettagli?"
    
    def reset(self):
        # Resettiamo lo stato del dialogo
        self.state = DialogueState()

## 6. Implementazione del Chatbot

Ora mettiamo insieme tutti i componenti per creare il nostro chatbot completo.

In [ ]:
# Classe principale del chatbot
class CustomerCareBot:
    def __init__(self, intents_data):
        self.name = "TelcoAssistant"
        self.dialogue_manager = DialogueManager(intents_data)
    
    def respond_to(self, user_message):
        # Processiamo il messaggio e otteniamo una risposta
        response = self.dialogue_manager.process(user_message)
        return response
    
    def start_new_conversation(self):
        # Resettiamo lo stato del dialogo
        self.dialogue_manager.reset()
        # Messaggio di benvenuto
        welcome_message = "Ciao! Sono l'assistente virtuale di TelcoService. Come posso aiutarti oggi?"
        self.dialogue_manager.state.add_bot_response(welcome_message)
        return welcome_message
    
    def get_conversation_history(self):
        # Restituiamo la storia della conversazione
        return self.dialogue_manager.state.conversation_history

In [ ]:
# Funzione per visualizzare la conversazione
def display_conversation(conversation_history):
    for turn in conversation_history:
        if turn["role"] == "user":
            print(f"\033[94mUtente: {turn['message']}\033[0m")
        else:
            print(f"\033[92mBot: {turn['message']}\033[0m")
        print()

In [ ]:
# Creiamo un'istanza del chatbot
bot = CustomerCareBot(intents_data)

# Iniziamo una nuova conversazione
welcome_message = bot.start_new_conversation()
print(f"\033[92mBot: {welcome_message}\033[0m\n")

In [ ]:
# Simuliamo una conversazione
conversation = [
    "Buongiorno, ho un problema con la mia connessione internet",
    "Ho già provato a riavviare il router ma non funziona",
    "Sì, i cavi sono collegati correttamente e nessun altro dispositivo riesce a connettersi",
    "Vorrei parlare con un operatore",
    "Si tratta di un problema tecnico",
    "Grazie mille per l'assistenza",
    "Arrivederci"
]

for message in conversation:
    print(f"\033[94mUtente: {message}\033[0m")
    response = bot.respond_to(message)
    print(f"\033[92mBot: {response}\033[0m\n")

In [ ]:
# Visualizziamo lo stato finale del dialogo
print(f"Stato finale del dialogo: {bot.dialogue_manager.state}")

## 7. Simulazione di Scenari di Customer Care

Testiamo il nostro chatbot con alcuni scenari realistici di customer care.

In [ ]:
# Funzione per simulare una conversazione completa
def simulate_conversation(bot, scenario_name, messages):
    print(f"\033[95m=== Scenario: {scenario_name} ===\033[0m\n")
    
    # Iniziamo una nuova conversazione
    welcome_message = bot.start_new_conversation()
    print(f"\033[92mBot: {welcome_message}\033[0m\n")
    
    # Processiamo ogni messaggio
    for message in messages:
        print(f"\033[94mUtente: {message}\033[0m")
        response = bot.respond_to(message)
        print(f"\033[92mBot: {response}\033[0m\n")
    
    print("\033[95m=== Fine Scenario ===\033[0m\n")

In [ ]:
# Scenario 1: Problema di connessione
scenario1 = [
    "La mia connessione internet non funziona da ieri sera",
    "Ho già provato a riavviare il router ma continua a non funzionare",
    "I cavi sembrano tutti collegati correttamente",
    "Nessun dispositivo riesce a connettersi, né via wifi né via cavo",
    "Vorrei parlare con un tecnico per risolvere il problema",
    "Si tratta di un problema con la fibra ottica",
    "Grazie per l'assistenza"
]

simulate_conversation(bot, "Problema di Connessione", scenario1)

In [ ]:
# Scenario 2: Informazioni sulla fatturazione
scenario2 = [
    "Ho ricevuto una fattura con un importo strano",
    "L'importo è di 75€",
    "Sì, preferirei parlare con un operatore del servizio clienti",
    "Si tratta di un problema di fatturazione",
    "Grazie mille"
]

simulate_conversation(bot, "Informazioni sulla Fatturazione", scenario2)

In [ ]:
# Scenario 3: Cambio piano tariffario
scenario3 = [
    "Vorrei informazioni sui vostri piani tariffari",
    "Mi interessa il Piano Premium, cosa include esattamente?",
    "E c'è un vincolo di permanenza?",
    "Grazie per le informazioni, ci penserò",
    "Arrivederci"
]

simulate_conversation(bot, "Cambio Piano Tariffario", scenario3)

## 8. Valutazione del Chatbot

Valutiamo le performance del nostro chatbot su diversi aspetti.

In [ ]:
# Funzione per valutare il chatbot su un set di test
def evaluate_chatbot(bot, test_cases):
    results = []
    
    for case in test_cases:
        # Resettiamo il bot per ogni caso di test
        bot.start_new_conversation()
        
        # Processiamo il messaggio
        message = case['message']
        expected_intent = case['expected_intent']
        
        # Otteniamo la risposta e l'intento riconosciuto
        bot.respond_to(message)
        actual_intent = bot.dialogue_manager.state.current_intent
        
        # Verifichiamo se l'intento è corretto
        is_correct = actual_intent == expected_intent
        
        results.append({
            'message': message,
            'expected_intent': expected_intent,
            'actual_intent': actual_intent,
            'is_correct': is_correct
        })
    
    # Calcoliamo l'accuratezza
    accuracy = sum(1 for r in results if r['is_correct']) / len(results)
    
    return {
        'results': results,
        'accuracy': accuracy
    }

In [ ]:
# Creiamo un set di test
test_cases = [
    {'message': 'Ciao, come stai?', 'expected_intent': 'saluto'},
    {'message': 'Buongiorno, avrei bisogno di assistenza', 'expected_intent': 'saluto'},
    {'message': 'La mia connessione non funziona', 'expected_intent': 'problema_connessione'},
    {'message': 'Internet è molto lento ultimamente', 'expected_intent': 'problema_connessione'},
    {'message': 'Quando arriverà il mio ordine?', 'expected_intent': 'stato_ordine'},
    {'message': 'Vorrei tracciare la mia spedizione', 'expected_intent': 'stato_ordine'},
    {'message': 'Ho una domanda sulla mia ultima bolletta', 'expected_intent': 'informazioni_fatturazione'},
    {'message': 'C\'è un addebito che non riconosco', 'expected_intent': 'informazioni_fatturazione'},
    {'message': 'Vorrei passare a un piano con più giga', 'expected_intent': 'cambio_piano'},
    {'message': 'Quali sono le vostre offerte attuali?', 'expected_intent': 'cambio_piano'},
    {'message': 'Il mio telefono non si connette al wifi', 'expected_intent': 'assistenza_tecnica'},
    {'message': 'Come configuro la mail sul nuovo smartphone?', 'expected_intent': 'assistenza_tecnica'},
    {'message': 'A che ora chiude il negozio?', 'expected_intent': 'orari_negozi'},
    {'message': 'Dove si trova il punto vendita più vicino?', 'expected_intent': 'orari_negozi'},
    {'message': 'Voglio parlare con una persona vera', 'expected_intent': 'contatto_operatore'},
    {'message': 'Passami a un operatore umano', 'expected_intent': 'contatto_operatore'},
    {'message': 'Grazie mille per l\'aiuto', 'expected_intent': 'ringraziamento'},
    {'message': 'Ti ringrazio, sei stato molto utile', 'expected_intent': 'ringraziamento'},
    {'message': 'Arrivederci', 'expected_intent': 'arrivederci'},
    {'message': 'Ciao ciao, alla prossima', 'expected_intent': 'arrivederci'}
]

# Valutiamo il chatbot
evaluation = evaluate_chatbot(bot, test_cases)

# Visualizziamo i risultati
print(f"Accuratezza complessiva: {evaluation['accuracy']:.2%}\n")

# Creiamo un DataFrame per una migliore visualizzazione
results_df = pd.DataFrame(evaluation['results'])
results_df

In [ ]:
# Visualizziamo l'accuratezza per intento
intent_accuracy = results_df.groupby('expected_intent')['is_correct'].mean().sort_values()

plt.figure(figsize=(10, 6))
intent_accuracy.plot(kind='barh')
plt.title('Accuratezza per Intento')
plt.xlabel('Accuratezza')
plt.ylabel('Intento')
plt.xlim(0, 1)
plt.grid(axis='x')

# Aggiungiamo le etichette con le percentuali
for i, acc in enumerate(intent_accuracy):
    plt.text(acc + 0.01, i, f"{acc:.0%}", va='center')

plt.tight_layout()
plt.show()

## 9. Interfaccia Utente Semplice

Creiamo una semplice interfaccia utente per interagire con il chatbot.

In [ ]:
# Funzione per interagire con il chatbot
def chat_with_bot():
    # Creiamo un'istanza del chatbot
    bot = CustomerCareBot(intents_data)
    
    # Iniziamo una nuova conversazione
    welcome_message = bot.start_new_conversation()
    print(f"\033[92mBot: {welcome_message}\033[0m\n")
    
    # Loop di conversazione
    while True:
        # Otteniamo l'input dell'utente
        user_input = input("Tu: ")
        
        # Controlliamo se l'utente vuole uscire
        if user_input.lower() in ['exit', 'quit', 'bye', 'arrivederci', 'esci']:
            print("\n\033[92mBot: Grazie per aver utilizzato il nostro servizio. Arrivederci!\033[0m")
            break
        
        # Otteniamo la risposta del bot
        response = bot.respond_to(user_input)
        print(f"\n\033[92mBot: {response}\033[0m\n")

# Eseguiamo la funzione per interagire con il chatbot
# Nota: Decommentare la riga seguente per avviare la chat interattiva
# chat_with_bot()

## 10. Conclusioni e Considerazioni Finali

In questo notebook, abbiamo sviluppato un chatbot per il customer care utilizzando tecniche di Natural Language Processing. Abbiamo:

1. Implementato un sistema di comprensione del linguaggio naturale (NLU) per riconoscere gli intenti degli utenti
2. Creato un estrattore di entità per identificare informazioni rilevanti nei messaggi
3. Sviluppato un gestore di dialogo per mantenere lo stato della conversazione
4. Implementato un sistema di generazione di risposte basato sugli intenti e sul contesto
5. Testato il chatbot con scenari realistici di customer care
6. Valutato le performance del chatbot su diversi aspetti

### Considerazioni per l'Implementazione Reale

In un contesto aziendale reale, ci sono ulteriori considerazioni da tenere in mente:

- **Integrazione con i Sistemi Esistenti**: Il chatbot dovrebbe integrarsi con i sistemi CRM, di ticketing e di gestione degli ordini per accedere a informazioni in tempo reale.

- **Personalizzazione**: Il chatbot dovrebbe essere in grado di personalizzare le risposte in base alla storia del cliente, alle preferenze e al contesto.

- **Multilingualità**: Per aziende che operano in mercati internazionali, il supporto per multiple lingue è essenziale.

- **Modelli più Avanzati**: Per casi d'uso più complessi, si potrebbero considerare modelli pre-addestrati come BERT o GPT per una comprensione più profonda del linguaggio.

- **Apprendimento Continuo**: Implementare meccanismi per il miglioramento continuo del chatbot basato sui feedback degli utenti e sulle conversazioni passate.

- **Handover a Operatori Umani**: Definire criteri chiari per quando il chatbot dovrebbe passare la conversazione a un operatore umano.

- **Monitoraggio e Analytics**: Implementare strumenti per monitorare le performance del chatbot, identificare aree di miglioramento e misurare l'impatto sul business.

- **Privacy e Sicurezza**: Assicurarsi che il chatbot gestisca i dati sensibili in modo sicuro e conforme alle normative sulla privacy.

Un chatbot ben progettato può migliorare significativamente l'esperienza del cliente, ridurre i costi operativi e liberare gli operatori umani per gestire casi più complessi che richiedono empatia e giudizio critico.